In [1]:
import json
import joblib
import warnings
import numpy as np
import pandas as pd
import random as rd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline

from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
warnings.filterwarnings('ignore')

In [3]:
PATH = '.../NeuralRTE/'

MC_data = pd.read_csv(PATH+'data/MC_data.csv', header=0)
MC_data['Rd_r'] = MC_data.apply(lambda r : json.loads(r['Rd_r']), axis=1)
MC_data = MC_data.drop(columns="Rd")
MC_data

,mua,mus,g,Rd_r
0,1.0,10,0.00,"[442.3838806152, 129.3646697998, 71.6296768188..."
1,1.0,10,0.01,"[432.5369873047, 126.4318695068, 70.0508193969..."
2,1.0,10,0.02,"[420.2418518066, 124.2272491455, 68.1100692749..."
3,1.0,10,0.03,"[406.6548156738, 122.1433410645, 66.1495437622..."
4,1.0,10,0.04,"[394.990814209, 119.5891113281, 64.6467666626,..."
...,...,...,...,...
828095,10.0,1000,0.86,"[1852.970214844, 624.1881713867, 387.926788330..."
828096,10.0,1000,0.87,"[1696.877441406, 569.4912719727, 351.208251953..."
828097,10.0,1000,0.88,"[1541.02331543, 516.1333007812, 318.9991455078..."
828098,10.0,1000,0.89,"[1390.891479492, 461.7012329102, 282.824951171..."


In [4]:
r_range = [0.00133,0.00311,0.00507,0.00705,0.00904,0.01103,0.01303,0.01502,0.01702,0.01902,0.02102,0.02301,0.02501,
           0.02701,0.02901,0.03101,0.03301,0.03501,0.03701,0.03901,0.04101,0.04301,0.04501,0.04701,0.04901,0.05101,
           0.05301,0.05501,0.05701,0.05901,0.06101,0.06301,0.06501,0.067,0.069,0.071,0.073,0.075,0.077,0.079,0.081,
           0.083,0.085,0.087,0.089,0.091,0.093,0.095,0.097,0.099,0.101,0.103,0.105,0.107,0.109,0.111,0.113,0.115,
           0.117,0.119,0.121,0.123,0.125,0.127,0.129,0.131,0.133,0.135,0.137,0.139,0.141,0.143,0.145,0.147,0.149,
           0.151,0.153,0.155,0.157,0.159,0.161,0.163,0.165,0.167,0.169,0.171,0.173,0.175,0.177,0.179,0.181,0.183,
           0.185,0.187,0.189,0.191,0.193,0.195,0.197,0.199,0.201]

In [5]:
# Final models
regressors = [LinearRegression(fit_intercept=True, positive=False, n_jobs=-1), 
              MLPRegressor(max_iter=100, hidden_layer_sizes=(100), solver='adam', learning_rate='constant', activation='relu', batch_size=200, alpha=1e-05), 
              DecisionTreeRegressor(criterion='friedman_mse', splitter='best', min_samples_split=2, min_samples_leaf=1, max_features=1.0, max_depth=20), 
              RandomForestRegressor(n_estimators=50, min_samples_split=3, min_samples_leaf=1, max_features=1.0, max_depth=20, bootstrap=True, n_jobs=-1)] #High memory requirement (reduce n_estimators or max_depth to run)
names = ['LinearRegression', 'MLPRegressor', 'DecisionTreeRegressor', 'RandomForestRegressor']

<br>

# Functions:

In [6]:
def printTb(df):
    print(tabulate(df, headers = 'keys', tablefmt = 'fancy_grid', numalign = 'center', floatfmt=".6f"))

In [8]:
def reconProfile(M1_p,M2_p,M3_p, df, i1, i2, i3):
    fig, ax = plt.subplots(figsize=(20, 12), dpi=600)

    m1_p1 = M1_p[i1];  m1_p2 = M1_p[i2];  m1_p3 = M1_p[i3]
    m2_p1 = M2_p[i1];  m2_p2 = M2_p[i2];  m2_p3 = M2_p[i3]
    m3_p1 = M3_p[i1];  m3_p2 = M3_p[i2];  m3_p3 = M3_p[i3]
    g1_t = df.iloc[i1,-1]; g2_t = df.iloc[i2,-1]; g3_t = df.iloc[i3,-1]

    # Model 1
    ax.semilogy(r_range, m1_p1, color="black", linestyle="dashed", marker='o', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m1_p2, color="black", linestyle="dashed", marker='o', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m1_p3, color="black", linestyle="dashed", marker='o', markersize=4.25, linewidth=0.65)

    # Model 2
    ax.semilogy(r_range, m2_p1, color="black", linestyle="dashed", marker='^', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m2_p2, color="black", linestyle="dashed", marker='^', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m2_p3, color="black", linestyle="dashed", marker='^', markersize=4.25, linewidth=0.65)

    # Model 3
    ax.semilogy(r_range, m3_p1, color="black", linestyle="dashed", marker='s', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m3_p2, color="black", linestyle="dashed", marker='s', markersize=4.25, linewidth=0.65)
    ax.semilogy(r_range, m3_p3, color="black", linestyle="dashed", marker='s', markersize=4.25, linewidth=0.65)


    ax.semilogy(r_range, g1_t, color="red"  , linestyle="solid", linewidth=1.2)
    ax.semilogy(r_range, g2_t, color="green", linestyle="solid", linewidth=1.2)
    ax.semilogy(r_range, g3_t, color="blue" , linestyle="solid", linewidth=1.2)

    ax.set_xlabel('$r$'+' '+'$[cm]$', fontsize = 45, labelpad=20)
    ax.set_ylabel('$Rd$', fontsize = 45, rotation = 0, labelpad=30)
    ax.tick_params(axis='both', which='major', labelsize=25)
    ax.set_xlim(0,0.195)

    g_NN   = Line2D([0], [0], label='ML (NN)', color='black', markersize=8, marker='o', linestyle=':')
    g_DT   = Line2D([0], [0], label='ML (DT)', color='black', markersize=8, marker='^', linestyle=':')
    g_RF   = Line2D([0], [0], label='ML (RF)', color='black', markersize=8, marker='s', linestyle=':')
    g_0_GT = Line2D([0], [0], label='g = 0.0, MC', color='darkred', markersize=14, linestyle='-')
    g_5_GT = Line2D([0], [0], label='g = 0.5, MC', color='darkgreen', markersize=15, linestyle='-')
    g_8_GT = Line2D([0], [0], label='g = 0.8, MC', color='darkblue', markersize=17, linestyle='-')

    plt.legend(handles=[g_NN,g_DT,g_RF,g_0_GT,g_5_GT,g_8_GT], loc='upper right', fontsize="18")

    # plt.savefig(PATH+'figures/profile_reconstruction_plot.pdf')
    plt.show()

In [9]:
# A function for evaluating the perfomance of a model on train + test datasets
def evaluate(model, data): 
    X   = data[0]; Y   = data[1]
    X_t = data[2]; Y_t = data[3]

    S = dt.datetime.now()
    model.fit(X,Y)
    Y_p = model.predict(X_t)
    E = dt.datetime.now()
    DUR = round((E-S).total_seconds(), 3)
    
    #Getting metrics
    MSE  = mean_squared_error(Y_t,Y_p)
    RMSE = np.sqrt(mean_squared_error(Y_t,Y_p))
    MAE  = mean_absolute_error(Y_t,Y_p)
    R2  = r2_score(Y_t,Y_p)
    
    out = pd.DataFrame([{'MSE': MSE, 'RMSE': RMSE, 'MAE': MAE, 'R2': R2, 'Time(s)': DUR}])
    return out, model

<br>

# Evaluation:

In [10]:
# Spliting our data for training (80%) and testing (20%)
train_set, test_set = train_test_split(MC_data.copy(), test_size=0.2)
train_set = train_set.reset_index(drop = True)
test_set = test_set.reset_index(drop = True)

In [11]:
#Getting X/y_train/test and apply StandardScaler() on the inputs data (X)
X = train_set.iloc[:,:-1]
Y = np.array(list(train_set.iloc[:, -1]))
X = StandardScaler().fit_transform(X)
    
X_t = test_set.iloc[:,:-1]
Y_t = np.array(list(test_set.iloc[:, -1]))
X_t = StandardScaler().fit_transform(X_t)

data = [X,Y,X_t,Y_t]

In [12]:
#Going through every regressor and evaluate its performance
lr_df, lr_model = evaluate(regressors[0], data)
nn_df, nn_model = evaluate(regressors[1], data)
dt_df, dt_model = evaluate(regressors[2], data)
rf_df, rf_model = evaluate(regressors[3], data)

In [13]:
RDR_DATA = lr_df
RDR_DATA = pd.concat([RDR_DATA, nn_df], axis = 0)
RDR_DATA = pd.concat([RDR_DATA, dt_df], axis = 0)
RDR_DATA = pd.concat([RDR_DATA, rf_df], axis = 0)
RDR_DATA = RDR_DATA.set_axis(names, axis = 0)

In [14]:
printTb(RDR_DATA.sort_values(['MAE', 'RMSE', 'R2']))

╒═══════════════════════╤══════════════╤════════════╤═══════════╤══════════╤════════════╕
│                       │     MSE      │    RMSE    │    MAE    │    R2    │  Time(s)   │
╞═══════════════════════╪══════════════╪════════════╪═══════════╪══════════╪════════════╡
│ RandomForestRegressor │   0.405366   │  0.636684  │ 0.129659  │ 0.996885 │ 61.162000  │
├───────────────────────┼──────────────┼────────────┼───────────┼──────────┼────────────┤
│ DecisionTreeRegressor │   1.812264   │  1.346204  │ 0.247986  │ 0.996144 │ 10.463000  │
├───────────────────────┼──────────────┼────────────┼───────────┼──────────┼────────────┤
│ MLPRegressor          │  140.336920  │ 11.846388  │ 1.849195  │ 0.801273 │ 290.336000 │
├───────────────────────┼──────────────┼────────────┼───────────┼──────────┼────────────┤
│ LinearRegression      │ 24410.736221 │ 156.239356 │ 18.357521 │ 0.469929 │  0.971000  │
╘═══════════════════════╧══════════════╧════════════╧═══════════╧══════════╧════════════╛


<br>

# Reconstuction:

In [15]:
# Test different combination of mua, mus, and g here:
id1 = MC_data.index[(MC_data.mua == 10) & (MC_data.mus == 90) & (MC_data.g == 0.0)][0]
id2 = MC_data.index[(MC_data.mua == 10) & (MC_data.mus == 90) & (MC_data.g == 0.5)][0]
id3 = MC_data.index[(MC_data.mua == 10) & (MC_data.mus == 90) & (MC_data.g == 0.8)][0]

In [16]:
test_data = StandardScaler().fit_transform(MC_data.iloc[:,:-1])

M1_p = nn_model.predict(test_data)
M2_p = dt_model.predict(test_data)
M3_p = rf_model.predict(test_data)

In [17]:
reconProfile(M1_p,M2_p,M3_p, MC_data, id1,id2,id3)

<br>

## Saving...

In [18]:
# joblib.dump(dt_model, "{0}/models/{1}.sav".format(PATH, 'rdr_model'))